In [2]:
import csv
import json
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
import numpy as np

response = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CR2016Geo.json?lang=E&geos=PR&topic=2&&notes=0')

b = json.loads(response.text[2:])
#pprint(b['DATA'])

prov = []
dguid=[]
for i in range((len(b['DATA']))):
    prov.append(b['DATA'][i][2])
    dguid.append(b['DATA'][i][0])

province_dguid = pd.DataFrame({'Province':prov,'dguid':dguid})
province_dguid

education = pd.DataFrame()
for id in province_dguid['dguid']:
    response2 = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json?lang=E&dguid='+id+'&topic=2&&notes=0')
    b2 = json.loads(response2.text[2:])
    #pprint(b2['DATA'])
    education = education.append(b2['DATA'])
    
education

ConnectionError: HTTPSConnectionPool(host='www12.statcan.gc.ca', port=443): Max retries exceeded with url: /rest/census-recensement/CR2016Geo.json?lang=E&geos=PR&topic=2&&notes=0 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001524D6EDE10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
education_n = education[education[10].str.contains('  No certificate, diploma or degree')]
education_s = education[education[10].str.contains('  Secondary (high) school diploma or equivalency certificate')]
education_p = education[education[10].str.contains('  Postsecondary certificate, diploma or degree')]
edu_n = pd.DataFrame({'Provinces':education_n[1],'No certificate, diploma or degree':education_n[13]}).set_index('Provinces')
edu_s = pd.DataFrame({'Provinces':education_s[1],'Aboriginal':education_s[13]}).set_index('Provinces')
edu_p = pd.DataFrame({'Provinces':education_p[1],'Aboriginal':education_p[13]}).set_index('Provinces')
edu = pd.merge(edu_n,edu_s, on='Provinces')
edu = pd.merge(edu,edu_p, on='Provinces')
edu

In [ ]:
edu = edu.groupby('Provinces')['Number of Income Recipients','Total Income'].sum
edu

In [7]:
education.to_csv("education.csv",encoding="utf-8", index=False, header=True)